In [117]:
%run my_import

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"
io = ZmsgIO(uri)
io.debug = False

def reply_cmd(cmd):
    reply = io.doit(cmd)
    yaml_msg = yaml.safe_load(reply['msg'])
    return yaml_msg

status_cmds = {
    'FAN_ON':           0x0026,
    'FAN_OFF':			0x0062,
    'LED_ON':           0x0019,
    'LED_OFF':          0x0091,
    'RELEASE_BRAKE':    0x00BD,
    'ENGAGE_BRAKE':     0x00DB,
}
reply_cmds = {
    'CMD_DONE':         0x7800,
    'CMD_WORKING':		0xD000,
    'CMD_ERROR':		0xAA00,
    'CMD_NOCOND':       0xBB00,
}

def ctrl_status_cmd(ctrl_cmd: int, bid: int):
    msg = reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'ctrl_status_cmd': ctrl_cmd}).set_bid(bid))
    msg = reply_cmd(SdoCmd(rd_sdo=['ctrl_status_cmd_ack'],wr_sdo={}).set_bid(bid))
    print(hex(msg['ctrl_status_cmd_ack']))
    return msg['ctrl_status_cmd_ack'] == ctrl_cmd + reply_cmds['CMD_DONE']

def ctrl_status_cmd_str(ctrl_cmd: str, bid: int):
    return ctrl_status_cmd(status_cmds[ctrl_cmd], bid) 

def sdo_filter(snames:list, sdos:dict):
    return {key:sdos[key] for key in snames}

/home/amargan/work/anaconda3/envs/repl/bin/python


In [118]:
scan_ids = reply_cmd(master_cmd_get_slave_descr).keys()
ids=list(scan_ids)
print(ids)
#assert(iD in ids)

[201, 69]


In [119]:
iD = 69
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))

In [120]:
sf =sdo_filter(['m3_fw_ver','c28_fw_ver','hw_config_low','hw_config_high'],sdo_dict)
print(sf)
hex( (sf['hw_config_high']<<16) + sf['hw_config_low'] )

{'m3_fw_ver': '220419xx', 'c28_fw_ver': 21122115, 'hw_config_low': 21527, 'hw_config_high': 21248}


'0x53005417'

In [73]:
# release all ids brakes using aux_pdo command
for iD in ids :
    reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_cmd_control': 1.0}).set_bid(iD))
    time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_cmd_status'],wr_sdo={}).set_bid(iD))['master_command_pdo_ack']
    print(iD, ret)

201 
69 51.0


In [7]:
# engage all ids brakes using aux_pdo command
for iD in ids :
    reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_command_pdo': 2.0}).set_bid(iD))
    time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_command_pdo_ack'],wr_sdo={}).set_bid(iD))['master_command_pdo_ack']
    print(iD, ret)

201 
69 52.0


In [81]:
sdo = ['torque','motor_pos','link_pos','Min_pos','Max_pos','Max_cur']
sdo_link = ['link_pos','fault','linkEncBadReadPPM','absLinkEncErr']
for i in range(2):
    G = io.doit4ids([iD],SdoCmd(rd_sdo=sdo,wr_sdo={}))
    #[g for g in G ]
    for g in G :
        print(g['msg'])
    time.sleep(0.2)

type: SLAVE_SDO_CMD
slave_sdo_cmd {
  board_id: 69
  rd_sdo: "torque"
  rd_sdo: "motor_pos"
  rd_sdo: "link_pos"
  rd_sdo: "Min_pos"
  rd_sdo: "Max_pos"
  rd_sdo: "Max_cur"
}

{Max_cur: 10.000000, Max_pos: 2.891593, Min_pos: -2.891593, link_pos: 0.613197, motor_pos: 0.614030, torque: 8.942866}
type: SLAVE_SDO_CMD
slave_sdo_cmd {
  board_id: 69
  rd_sdo: "torque"
  rd_sdo: "motor_pos"
  rd_sdo: "link_pos"
  rd_sdo: "Min_pos"
  rd_sdo: "Max_pos"
  rd_sdo: "Max_cur"
}

{Max_cur: 10.000000, Max_pos: 2.891593, Min_pos: -2.891593, link_pos: 0.613197, motor_pos: 0.614030, torque: 9.103990}


In [18]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0x3B))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [25]:
io.doit(ctrl_cmd_stop.set_bid(iD))

{'ctrl_cmd': {'type': 'CTRL_CMD_STOP', 'board_id': 69, 'value': 0.0, 'gains': {}}, 'type': 'CTRL_CMD'}
type: CTRL_CMD
ctrl_cmd {
  type: CTRL_CMD_STOP
  board_id: 69
  value: 0.0
}



{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [20]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [115]:
io.debug=0
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 250
gains['kd'] = 5
#gains['ki'] = 0
gains['kkp'] = 2
gains['kkd'] = 0.001
gains['kki'] = 0.08
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(*gains.values()))
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',2000,10,2,0.001,0))
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',0,0,5,0.001,0.08))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [116]:
io.doit(ctrl_cmd_stop.set_bid(iD))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [ ]:
for iD in ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', iD)

In [ ]:
for iD in ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', iD)

In [ ]:
ctrl_status_cmd_str('FAN_ON', iD)

In [ ]:
ctrl_status_cmd_str('FAN_OFF', iD)

In [80]:
io.doit(SdoCmd(rd_sdo=[],wr_sdo={'Max_cur': 10}).set_bid(iD))

{'slave_sdo_cmd': {'board_id': 69, 'rd_sdo': [], 'wr_sdo': [{'name': 'Max_cur', 'value': '10'}]}, 'type': 'SLAVE_SDO_CMD'}
type: SLAVE_SDO_CMD
slave_sdo_cmd {
  board_id: 69
  wr_sdo {
    name: "Max_cur"
    value: "10"
  }
}



{'type': 1, 'cmd_type': 7, 'msg': '{}'}

In [82]:
io.doit(flash_cmd_save_flash.set_bid(iD))

{'flash_cmd': {'type': 'SAVE_PARAMS_TO_FLASH', 'board_id': 69}, 'type': 'FLASH_CMD'}
type: FLASH_CMD
flash_cmd {
  type: SAVE_PARAMS_TO_FLASH
  board_id: 69
}



{'type': 1, 'cmd_type': 3, 'msg': 'Ok !!!'}